In [1]:
from transformers import BertModel, BertTokenizer
import torch
from torch.utils.tensorboard import SummaryWriter
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer, util

C:\Users\kevin\anaconda3\envs\chatgpt\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Design prompt 3, human prompts
human = []
with open("amazonTurkDesPrompt3.csv", 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        human.append(row[0])
human = [i.replace('\n',' ') for i in human]    # Removing the line breaks as they mess with tensorboard.
human

['ï»¿A jump rope that is also to be used as a tie and or a belt during the day and while at work.',
 'A muscle stimulate band that can be worn over any area of the body that promotes muscle use, great for traveling and people who have to sit a lot.',
 'A simple, but strong band that has a hook to be able to attach to any (thin) pole or sturdy object. The band will have a handle on one side to do pulling exercises for lats, curls for biceps, chest flys for pecs, arm extentions for triceps, etc. It will be versatile and lightweight, and can be used almost anywhere.',
 'A calf strengthener, similar to a finger strengthener. It is placed on the floor of the car and pressed and released like a spring. More resistance could be added via a stiffer spring for a more difficult excercise.',
 'A rubber ball that can be inflated / deflated extremely quickly.',
 'An x shaped resistance band that has 2 hand grips and 2 foot holds. The foothold will be stepped on to secure the system down, then the o

In [3]:
# Design prompt 3, gpt prompts
desprompt3 = pd.read_csv('DesPrompt3_100Soln.csv', sep = ",")
lst = []
for d in desprompt3.values:
    lst.append(d[0])
gpt = []
for i in range(len(lst)):
    mod_statements = lst[i].replace(lst[i][:3], '')
    gpt.append(mod_statements)

In [4]:
# embed data
model = SentenceTransformer('all-MiniLM-L6-v2')
gpt_embeddings = model.encode(gpt)
human_embeddings = model.encode(human)

In [5]:
gpt_embeddings

array([[-0.06169147,  0.02120759, -0.01162963, ..., -0.0548689 ,
         0.00779079,  0.06648231],
       [-0.02125286,  0.08223509, -0.06445835, ..., -0.06688277,
        -0.02493583,  0.06444902],
       [-0.01249652,  0.06014684,  0.01583032, ..., -0.04636282,
         0.02316219,  0.07209936],
       ...,
       [-0.0206801 ,  0.0367028 , -0.0532451 , ..., -0.0532966 ,
        -0.03527846,  0.06081413],
       [-0.03582376,  0.03189288, -0.07727049, ..., -0.11564801,
        -0.0564786 ,  0.00944735],
       [-0.03191879,  0.06474904, -0.05040888, ..., -0.01713385,
        -0.02701618,  0.04668273]], dtype=float32)

In [6]:
bert_output_human = torch.from_numpy(human_embeddings)
bert_output_gpt = torch.from_numpy(gpt_embeddings)

In [7]:
bert_output_gpt

tensor([[-0.0617,  0.0212, -0.0116,  ..., -0.0549,  0.0078,  0.0665],
        [-0.0213,  0.0822, -0.0645,  ..., -0.0669, -0.0249,  0.0644],
        [-0.0125,  0.0601,  0.0158,  ..., -0.0464,  0.0232,  0.0721],
        ...,
        [-0.0207,  0.0367, -0.0532,  ..., -0.0533, -0.0353,  0.0608],
        [-0.0358,  0.0319, -0.0773,  ..., -0.1156, -0.0565,  0.0094],
        [-0.0319,  0.0647, -0.0504,  ..., -0.0171, -0.0270,  0.0467]])

In [8]:
# visualize embeddings
writer = SummaryWriter('embeddings')
metadata = [('human', line) for line in human] + [('gpt', line) for line in gpt]
writer.add_embedding(torch.cat((bert_output_human, bert_output_gpt), 0), metadata=metadata, metadata_header=['writer', 'prompt'])

Run in terminal: tensorboard --logdir="embeddings\" --host localhost http://localhost:6006/ to open Tensorboard Refresh page.
Note when you run from anaconda, conda activate chatgpt -> cd chatgpt -> run the terminal command above.